In [1]:
from cartopy import config
import cartopy.crs as ccrs
import matplotlib.colors as plt_cols

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time, json, requests, pprint, copy, math, dateutil
from datetime import datetime, timedelta
from argovisHelpers import helpers as avh

import os

from Argovis_tasks_helpers import get_route

/Users/dgiglio/opt/miniconda3/envs/argovis_demos/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
API_KEY=''

### This will be updated with an API query
Argovis supports several different data sets with the API and data structures described here. They and their corresponding routes are:

 - Argo profiling float data, `/argo`
 - CCHDO ship-based profile data, `/cchdo`
 - tropical cyclone data from HURDAT and JTWC, `/tc`
 - Global Drifter Program data, `/drifters`
 - Easy Ocean, `/easyocean`
 - several gridded products:
   - Roemmich-Gilson total temperature and salinity, `/grids/rg09`
   - ocean heat content, `/grids/kg21`
   - GLODAP, `/grids/glodap`
 - Argone Argo float position forecast model data, `/argone`
 - Argo trajectory data, `/argotrajectories`
 - several satellite-based timeseries:
   - NOAA sea surface temperature, `/timeseries/noaasst`
   - Copernicus sea surface height, `/timeseries/copernicussla`
   - CCMP wind vector product, `/timeseries/ccmpwind`

In [3]:
# API call showing collections available for each route

#### parameters the user needs to select
# user selects list of collections
collections = ['argo', 'cchdo', 'drifters', 
               'easyocean', 'grids/rg09', '/grids/kg21',
               '/grids/glodap', '/argotrajectories', 
               '/timeseries/noaasst', '/timeseries/copernicussla',
               '/timeseries/ccmpwind', 'tc', 'extended/ar', 
              ]

# time period of interest for the API query
startDate = '2015-01-01T00:00:00Z'
endDate   = '2015-03-01T00:00:00Z'

# region to query could be added here

# long, lat grid for the output map
dx = 1
dy = 1
x_edges = np.arange(-180,181,dx)
y_edges = np.arange(-90,91,dy)

# verticalRange will be used only if it makes sense for that collection
verticalRange = '4000,4100'

In [4]:
def map_count_in_bins(name,x_edges,y_edges,dx,dy,fpath,startDate,endDate,vmin_map=1,vmax_map=1000):
    d = globals()[name] 
    lons = d['longitudes']#[x[1] for x in d]
    lats  = d['latitudes']#[x[2] for x in d]
    datetimes  = d['datetimes'] #[datetime.strptime(x[3][0:18]+'Z',"%Y-%m-%dT%H:%M:%SZ") for x in d]
    
    fig = plt.figure(figsize=(21, 7))
    
    h = np.histogram2d(lons, lats,[x_edges, y_edges])
    ax = plt.axes(projection=ccrs.PlateCarree())

    plt.pcolormesh(x_edges[0:-1]+dx/2, y_edges[0:-1]+dy/2, h[0].transpose(),  #60,
             transform=ccrs.PlateCarree(),norm=plt_cols.LogNorm(vmin=vmin_map, vmax=vmax_map ))

    ax.stock_img()
    ax.coastlines()
    
    plt.colorbar()

    plt.savefig(fpath+name+startDate[0:10]+'_'+endDate[0:10]+'.png')
    plt.show()

In [5]:
def create_df_from_minimal_query(mydict_name,api_output):
    bfr = pd.DataFrame(globals()[api_output])
    
    bfr.rename( columns={0 :'id'}, inplace=True )
    bfr.rename( columns={1 :'longitudes'}, inplace=True )
    bfr.rename( columns={2 :'latitudes'}, inplace=True )
    bfr.rename( columns={3 :'datetimes'}, inplace=True )
    bfr.rename( columns={4 :'source'}, inplace=True )
    return bfr

In [6]:
# define params for the query
params = {
        'startDate': startDate,
        'endDate': endDate,
        'compression': 'minimal'
    }

In [ ]:
# for each collection of interest, plot a map
for icollection in collections:
    print(icollection)
    iroute = get_route(icollection)
    print(iroute)
    iparams = params
    #iparams['presRange'] = verticalRange
    globals()[icollection] = avh.query(icollection, options=iparams, apikey=API_KEY, apiroot=iroute)
    
    if not globals()[icollection]:
        print(icollection + ': no data in region/time period selected')
    else:
        # create dataframe with columns of interest
        mydict_name = 'df_'+icollection
        globals()[mydict_name] = create_df_from_minimal_query(mydict_name=mydict_name,api_output=icollection)
        print(globals()[mydict_name])

argo
https://argovis-api.colorado.edu/
                id  longitudes  latitudes                 datetimes  \
0      6901829_071   18.648623  38.485058  2015-01-30T23:55:59.999Z   
1      2901339_114   63.420000  16.761000  2015-01-30T23:55:32.999Z   
2      6901836_048    9.073675  42.889797  2015-01-30T23:52:00.000Z   
3      6901831_040   35.728392  42.845038  2015-01-30T23:50:59.999Z   
4      1901385_182 -138.439000 -54.018000  2015-01-30T23:37:12.995Z   
...            ...         ...        ...                       ...   
27166  5904153_059 -164.433000 -74.048000  2015-01-31T00:15:24.000Z   
27167  2902503_059  137.020000  18.049000  2015-01-31T00:06:37.000Z   
27168  2902541_108  145.487000  25.142000  2015-01-31T00:01:06.999Z   
27169  2902096_058   96.562000  -2.237000  2015-01-31T00:00:00.000Z   
27170  2902103_057   63.050000  -3.965000  2015-01-31T00:00:00.000Z   

            source             5  
0      [argo_core]  [6901829_m0]  
1      [argo_core]  [2901339_m1]  
2  

In [ ]:
type(argo)